In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
from tqdm import tqdm_notebook

# Initialization

In [2]:
def initialize_wrd_emb(vocab_size, emb_size):
    WRD_EMB = np.random.randn(vocab_size, emb_size) * 0.01
    return WRD_EMB

def initialize_dense(input_size, output_size):
    W = np.random.randn(output_size, input_size) * 0.01
    b = np.random.randn(output_size, 1) * 0.01
    return W, b

def initialize_parameters(vocab_size, emb_size):
    WRD_EMB = initialize_wrd_emb(vocab_size, emb_size)
    W, b = initialize_dense(emb_size, vocab_size)
    
    parameters = {}
    parameters['WRD_EMB'] = WRD_EMB
    parameters['W'] = W
    parameters['b'] = b
    
    return parameters

# Forward Propagation

In [3]:
def ind_to_word_vecs(inds, parameters):
    """
    inds -- shape: (CBOW_N, number of examples)
    """
    WRD_EMB = parameters['WRD_EMB']
    word_vecs = np.take(WRD_EMB, inds.T, axis=0).T
    
    assert(word_vecs.shape == (WRD_EMB.shape[1], inds.shape[0], inds.shape[1]))
    
    return word_vecs

def mean_(word_vecs):
    word_vecs_mean = np.mean(word_vecs, axis=1)
    word_vecs_mean = word_vecs_mean.reshape(word_vecs.shape[0], -1)
    
    assert(word_vecs_mean.shape == (word_vecs.shape[0], word_vecs.shape[2]))
    
    return word_vecs_mean

def linear_dense(word_vecs_mean, parameters):
    W, b = parameters['W'], parameters['b']
    Z = np.dot(W, word_vecs_mean) + b
    
    assert(Z.shape == (W.shape[0], word_vecs_mean.shape[1]))
    
    return W, b, Z

def softmax(Z):
    softmax_out = np.divide(np.exp(Z), np.sum(np.exp(Z), axis=0, keepdims=True) + 0.001)
    
    assert(softmax_out.shape == Z.shape)

    return softmax_out

def forward_propagation(inds, parameters):
    word_vecs = ind_to_word_vecs(inds, parameters)
    word_vecs_mean = mean_(word_vecs)
    W, b, Z = linear_dense(word_vecs_mean, parameters)
    softmax_out = softmax(Z)
    
    caches = {}
    caches['inds'] = inds
    caches['word_vecs'] = word_vecs
    caches['word_vecs_mean'] = word_vecs_mean
    caches['W'] = W
    caches['b'] = b
    caches['Z'] = Z
    
    return softmax_out, caches

# Cost Function

In [4]:
def cross_entropy(softmax_out, Y):
    m = softmax_out.shape[1]
    cost = -(1 / m) * np.sum(np.sum(Y * np.log(softmax_out + 0.001), axis=0, keepdims=True), axis=1)
    return cost

# Backward Propagation

In [5]:
def softmax_backward(Y, caches):
    Z = caches['Z']
    dL_dZ = Z - Y
    
    assert(dL_dZ.shape == Z.shape)
    
    return dL_dZ

def dense_backward(dL_dZ, caches):
    """
    Z = W * X + b
    """
    W = caches['W']
    b = caches['b']
    word_vecs_mean = caches['word_vecs_mean']
    m = word_vecs_mean.shape[1]
    
    dL_dW = (1 / m) * np.dot(dL_dZ, word_vecs_mean.T)
    dL_db = (1 / m) * np.sum(dL_dZ, axis=1, keepdims=True)
    dL_dword_vecs_mean = np.dot(W.T, dL_dZ)

    assert(W.shape == dL_dW.shape)
    assert(b.shape == dL_db.shape)
    assert(word_vecs_mean.shape == dL_dword_vecs_mean.shape)
    
    return dL_dW, dL_db, dL_dword_vecs_mean

def mean_backward(dL_dword_vecs_mean, caches):
    """
    ele_mean_i = (1 / n) * (ele_1 + ele_2 + ele_n)
    """
    word_vecs = caches['word_vecs']
    CBOW_N = word_vecs.shape[1]
    
#     dL_dword_vecs = (1 / m) * (1 / CBOW_N) * np.ones((dL_dword_vecs_mean.shape[0], CBOW_N)) *\
#         np.sum(dL_dword_vecs_mean, axis=1, keepdims=True)
        
    dL_dword_vecs = (1 / m) * (1 / CBOW_N) * np.sum(dL_dword_vecs_mean, axis=1, keepdims=True)
    assert((word_vecs.shape[0], 1) == dL_dword_vecs.shape)
    
    return dL_dword_vecs

def backward_propagation(Y, caches):
    dL_dZ = softmax_backward(Y, caches)
    dL_dW, dL_db, dL_dword_vecs_mean = dense_backward(dL_dZ, caches)
    dL_dword_vecs = mean_backward(dL_dword_vecs_mean, caches)
    
    gradients = dict()
    gradients['dL_dZ'] = dL_dZ
    gradients['dL_dW'] = dL_dW
    gradients['dL_db'] = dL_db
    gradients['dL_dword_vecs'] = dL_dword_vecs
    
    return gradients

def update_parameters(parameters, caches, gradients, learning_rate):
    CBOW_N = caches['inds'].shape[0]
    vocab_size, emb_size = parameters['WRD_EMB'].shape
    inds = caches['inds']
    WRD_EMB = parameters['WRD_EMB']
    dL_dword_vecs = gradients['dL_dword_vecs']
    m = inds.shape[-1]
    
    
#     updated_WRD_EMD = parameters['WRD_EMB'][inds.T, :] -\
#         learning_rate * gradients['dL_dword_vecs'].T.reshape(1, CBOW_N, -1)
    for i in range(m):
        WRD_EMB[inds[:, i], :] -= dL_dword_vecs.T * learning_rate
    
#     parameters['WRD_EMB'][inds.flatten(), :] = updated_WRD_EMD.reshape(-1, emb_size)
    parameters['W'] -= learning_rate * gradients['dL_dW']
    parameters['b'] -= learning_rate * gradients['dL_db']

In [6]:
from sys import getsizeof
import gc


def cbow_model(X, Y, vocab_size, emb_size, learning_rate, epochs, batch_size=256, parameters=None, print_cost=False):
    costs = []
    m = X.shape[1]
    if parameters is None:
        parameters = initialize_parameters(vocab_size, emb_size)

#     batch_inds = list(range(0, m, batch_size))
    for epoch in range(epochs):
#         np.random.shuffle(batch_inds)
        for b, i in enumerate(range(0, m, batch_size)):
            X_batch = X[:, i:i+batch_size]
            Y_batch = Y[:, i:i+batch_size]

            softmax_out, caches = forward_propagation(X_batch, parameters)
            gradients = backward_propagation(Y_batch, caches)
            update_parameters(parameters, caches, gradients, learning_rate)
            cost = cross_entropy(softmax_out, Y_batch)
#             if b % 100 == 0:
#                 print('epoch {}, {}/{} - Cost: {}'.format(epoch, b, 1000, np.squeeze(cost)))
#             gc.collect()
#             del X_batch
#             del Y_batch
#             del caches
#             del gradients
            
            
        costs.append(cost)
        if print_cost and epoch % 1000 == 0:
            print("Cost after epoch {}: {}".format(epoch, np.squeeze(cost)))
        
    return parameters

# Toy Data : the quick brown fox jumped over the lazy dog

In [8]:
text_toy = ['the','quick','brown','fox','jumped','over','the','lazy','dog']
id_to_word_toy = dict()
word_to_id_toy = dict()
i = 0
for token in text_toy:
    if token in word_to_id_toy:
        continue
    id_to_word_toy[i] = token
    word_to_id_toy[token] = i
    i += 1
window_size = 1
example_len = 2 * 1 + 1
X_toy = []
Y_toy = []

for i in range(len(text_toy) - example_len + 1):
    X_toy.extend([word_to_id_toy[word] for word in text_toy[i:i+1] + text_toy[i+2:i+3]])
    Y_toy.append(word_to_id_toy[text_toy[i+1]])
        
X_toy = np.array(X_toy)
X_toy = X_toy.reshape(-1, window_size * 2).T
Y_toy = np.array(Y_toy)
Y_toy = Y_toy.reshape(1, -1)

In [10]:
vocab_size = 8
m = X_toy.shape[-1]
emb_size = 6

Y_toy_one_hot = np.zeros((vocab_size, m))
Y_toy_one_hot[Y_toy.flatten(), np.arange(m)] = 1

In [13]:
parameters = cbow_model(X_toy, Y_toy_one_hot, vocab_size, emb_size, 0.0025, 100000, parameters=None, batch_size=126, print_cost=True)

Cost after epoch 0: 2.071669643729546
Cost after epoch 1000: 2.0561158641183903
Cost after epoch 2000: 2.054868062259784
Cost after epoch 3000: 2.0547102072415426
Cost after epoch 4000: 2.0546409362318405
Cost after epoch 5000: 2.0545789115204616
Cost after epoch 6000: 2.0545174883779818
Cost after epoch 7000: 2.0544561238145422
Cost after epoch 8000: 2.054394773562
Cost after epoch 9000: 2.0543334341167845
Cost after epoch 10000: 2.0542721053063957
Cost after epoch 11000: 2.054210787229377
Cost after epoch 12000: 2.0541494800050644
Cost after epoch 13000: 2.0540881837530143
Cost after epoch 14000: 2.054026898591212
Cost after epoch 15000: 2.053965624635827
Cost after epoch 16000: 2.053904362001086
Cost after epoch 17000: 2.0538431107991713
Cost after epoch 18000: 2.053781871140115
Cost after epoch 19000: 2.0537206431317077
Cost after epoch 20000: 2.053659426879404
Cost after epoch 21000: 2.0535982224862375
Cost after epoch 22000: 2.053537030052741
Cost after epoch 23000: 2.05347584967

In [14]:
WRD_EMB = parameters['WRD_EMB']

In [15]:
def find_top_n_similar(word, wrd_emb, n=10):
    id_ = word_to_id_toy[word]
    vec_word = wrd_emb[id_, :]
    norm_vec_word = np.linalg.norm(vec_word)
    cos_sim = np.dot(wrd_emb, vec_word.T) / (np.linalg.norm(wrd_emb, axis=1) * norm_vec_word)
    top_n_ind = np.argsort(cos_sim)[-n:][::-1]
    return [id_to_word_toy[id_] for id_ in top_n_ind]

In [16]:
find_top_n_similar('lazy', WRD_EMB)

['lazy', 'brown', 'dog', 'the', 'quick', 'jumped', 'over', 'fox']

# Reference
- https://github.com/ujhuyz0110/wrd_emb/blob/master/word2vec_cbow.ipynb